In [ ]:
import os
from ultralytics import YOLO

# parameters
MODEL_PATH = "/home/huangliang/CV-Practical-training/runs/detect/train2/weights/best.pt"
IMAGE_ROOT_PATH = "./images"
IMAGE_NAME = "test4.png"

# Load a model
model = YOLO(MODEL_PATH)  # load an official model
# results = model.predict("./images/test1.jpg", classes=[1, 2])
res = model.predict(os.path.join(IMAGE_ROOT_PATH, 'origin', IMAGE_NAME))
res[0].save(filename=os.path.join(IMAGE_ROOT_PATH, 'pred', IMAGE_NAME))
res[0].show()

In [35]:
import os
from matplotlib import pyplot as plt
import numpy as np
from ultralytics import YOLO
import cv2  # 导入OpenCV库

class_dic = [
'two_wheeler',  # 二轮车主体
'helmet',       # 佩戴头盔的头部
'without_helmet'# 未戴头盔的头部
]

class_BGR = [
    (255, 0, 0),    # 二轮车主体      蓝
    (0, 255, 255),  # 佩戴头盔的头部  黄
    (0, 0, 255)     # 未戴头盔的头部  红
]

# Load a model
MODEL_PATH = "/home/huangliang/CV-Practical-training/runs/detect/train2/weights/best.pt"
model = YOLO(MODEL_PATH)  # load an official model


image_path = '/home/huangliang/CV-Practical-training/test/images/origin/test2.png'
image = cv2.imread(image_path)

results = model(image)  # 输入图像路径

# 4. 计数检测到的物体
count = {}
for result in results:
    for obj in result.boxes:
        cls = int(obj.cls[0])  # 获取类别索引
        count[cls] = count.get(cls, 0) + 1  # 统计数量

text_lines = []
for k, v in count.items():
    text_lines.append(f'{class_dic[k]}: {v}')


for obj in results[0].boxes:
    x1, y1, x2, y2 = map(int, obj.xyxy[0])  # 获取边界框坐标
    cls = int(obj.cls[0])  # 获取类别索引

    # 绘制矩形框
    cv2.rectangle(image, (x1, y1), (x2, y2), class_BGR[cls], 3)

    text = f'{class_dic[cls]}'
    
    # 获取文本的宽度和高度
    (text_width, text_height), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 2, 2)

    # 计算背景矩形的坐标
    background_x1 = x1
    background_y1 = y1 - text_height - baseline - 10  # 文本上方留出一些空间
    background_x2 = x1 + text_width
    background_y2 = y1

    # 绘制红色背景矩形
    cv2.rectangle(image, (background_x1, background_y1), (background_x2, background_y2), class_BGR[cls], cv2.FILLED)

    # 在矩形上绘制文本
    cv2.putText(image, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 3)

    # 在左上角绘制需要的信息
    # 设置初始文本位置
    x = 50
    y_start = 200  # 第一行的y坐标

    # 绘制每一行文本
    for i, line in enumerate(text_lines):
        y = y_start + i * 90
        cv2.putText(image, line, (x, y), cv2.FONT_HERSHEY_COMPLEX, 3, (0, 0, 0), 3, cv2.LINE_AA)
    

    

cv2.imwrite('image_lossless.png', image, [cv2.IMWRITE_PNG_COMPRESSION, 0])


0: 384x640 2 two_wheelers, 1 helmet, 1 without_helmet, 14.3ms
Speed: 4.2ms preprocess, 14.3ms inference, 5.6ms postprocess per image at shape (1, 3, 384, 640)


True